Final Project

DataSet: CIFAR-10

Introduction:
The CIFAR-10 dataset (Canadian Institute for Advanced Research, 10 classes) is a subset of the Tiny Images dataset and consists of 60000 32x32 color images. The images are labelled with one of 10 mutually exclusive classes: airplane, automobile (but not truck or pickup truck), bird, cat, deer, dog, frog, horse, ship, and truck (but not pickup truck). There are 6000 images per class with 5000 training and 1000 testing images per class.

Dataset overview:
Classes in dataset:
1. airplane
2. automobile
3. bird
4. cat
5. deer
6. dog
7. frog
8. horse
9. ship
10. truck


Classifer:  Logistic Classifier



Importing the modules needed to train the logistic model on the CIFAR-10 dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV
from sklearn.calibration import calibration_curve
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import time
import pickle
import tensorflow as tf
from itertools import product
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
seed = 1234

ModuleNotFoundError: No module named 'numpy'

Loading the dataset to the assigned variables

In [ ]:
# def unpickle(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict

# def load_cifar10_data(data_dir):
#     # Load training data
#     X_train = []
#     y_train = []
#     for i in range(1, 6):
#         batch = unpickle(data_dir + '/data_batch_' + str(i))
#         X_train.append(batch[b'data'])
#         y_train.append(batch[b'labels'])
#     X_train = np.concatenate(X_train, axis=0)
#     y_train = np.concatenate(y_train, axis=0)

#     # Load test data
#     test_batch = unpickle(data_dir + '/test_batch')
#     X_test = test_batch[b'data']
#     y_test = np.array(test_batch[b'labels'])

#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=seed)

#     return X_train, y_train, X_val, y_val, X_test, y_test

# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# creating 80%/10%/10% split
# 80% for training, 10% for validation (dev set), 10% for test

X= np.concatenate((X_train, X_test))
y= np.concatenate((y_train, y_test))

print(X.shape)

# test set provided is already 20% of data
# getting 60% training and 20% val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, shuffle=True, random_state=seed)


In [ ]:
# data_dir = 'cifar-10-batches-py'
# X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10_data(data_dir)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
#pre processing
# flatten y to 1D
y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

# sub-sampling
# create smaller training set with first 13k images
# want a sub sampled training set that has more data than the isolated test set
# training set = 13,000 sub-sample
X_train = X_train[:13000]
y_train = y_train[:13000]

# reshape x
X_train = np.reshape(X_train, (X_train.shape[0], -1)).astype("float32")
X_val = np.reshape(X_val, (X_val.shape[0], -1)).astype("float32")
X_test = np.reshape(X_test, (X_test.shape[0], -1)).astype("float32")


# standardize data
# want to standardize data to a range of 0 to 1
# dividing by 255 because pixel values range from 0-255
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

# Convert target labels to one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_val_one_hot = to_categorical(y_val, num_classes=10)



print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)



In [ ]:
#training with only 2 layers
activation_func = ['relu', 'relu', 'sigmoid', 'relu','sigmoid', 'relu']
learning_rates = [0.0001, 0.001, 0.05, 0.01, 0.1, 0.5,]
weight1 = [200, 150, 128, 1024, 250, 512]
weight2 = [100, 256, 120, 512, 310, 245]
# weight3 = [10, 10, 25, 10, 40, 75]
epochs= [5, 50, 20, 100, 15, 10]
batch_sizes = [20, 10, 30, 32, 50, 45]


for i in range(6):
    model = Sequential()
    model.add(Dense(weight1[i], input_shape=(3072,), activation=activation_func[i]))
    model.add(Dense(weight2[i], activation=activation_func[i]))
    model.add(Dense(10, activation="softmax"))

    # train the model using SGD
    print("[INFO] training network...")
    sgd = SGD(learning_rates[i])
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
    H = model.fit(X_train, y_train, validation_data=(X_val, y_val_one_hot), epochs=epochs[i], batch_size=batch_sizes[i])


In [ ]:


# Set the hyperparameters to iterate over
penalty_options = ['l1', 'l2']
c_values = [10, 1.0, 0.1, 0.01, 0.001]

# Initialize variables to store the best hyperparameters and accuracy
best_accuracy = 0.0
best_hyperparameters = {}

# Initialize lists to store the error rates and corresponding hyperparameters
train_error_rates = []
val_error_rates = []
hyperparameter_combinations = []

# Iterate over the hyperparameter combinations
for penalty in penalty_options:
    for c in c_values:
        print(penalty, c)
        # Create the logistic regression model with the current hyperparameters
        logistic_model = LogisticRegression(penalty=penalty, C=c, solver='saga')
        
        # Fit the model to the training data
        logistic_model.fit(X_train, y_train)
        
        # Evaluate the model on the train set
        train_predictions = logistic_model.predict(X_train)
        train_accuracy = accuracy_score(y_train, train_predictions)
        train_error_rate = 1 - train_accuracy
        train_error_rates.append(train_error_rate)
        
        # Evaluate the model on the validation set
        val_predictions = logistic_model.predict(X_val)    
        val_accuracy = accuracy_score(y_val, val_predictions)
        val_error_rate = 1 - val_accuracy
        val_error_rates.append(val_error_rate)

        hyperparameter_combinations.append((penalty, c))
        
        # Check if this combination has better accuracy than the previous best
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_hyperparameters = {'penalty': penalty, 'c': c}

# Convert the hyperparameter combinations into strings for plotting
hyperparameter_combinations = [str(combination) for combination in hyperparameter_combinations]

# Print the best hyperparameters and accuracy
print("Best Hyperparameters:", best_hyperparameters)
print("Validation Accuracy:", best_accuracy)


In [ ]:
# Create a line plot of the error rates
plt.plot(hyperparameter_combinations, train_error_rates, marker='o', label='Train')
plt.plot(hyperparameter_combinations, val_error_rates, marker='o', label='Validation')
plt.xlabel('Hyperparameter Combinations')
plt.ylabel('Error Rate')
plt.title('Error Rate for Different Hyperparameter Combinations')
plt.xticks(rotation='vertical')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Validate the classifier
logistic_model = LogisticRegression(penalty="l2", C=0.001, solver='saga')
logistic_model.fit(X_train, y_train)

y_pred_train = logistic_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)
print("Confusion Matrix:")
confusionMatrix = confusion_matrix(y_train, y_pred_train)
confusionMatrix = ConfusionMatrixDisplay(confusion_matrix = confusionMatrix)
confusionMatrix.plot()
print("Precision:", precision_score(y_train, y_pred_train, average='macro'))
print("Recall:", recall_score(y_train, y_pred_train, average='macro'))

In [ ]:
# Validate the classifier on the validation set
y_pred_val = logistic_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Validation Accuracy:", val_accuracy)
print("Confusion Matrix:")
confusionMatrix = confusion_matrix(y_val, y_pred_val)
confusionMatrix = ConfusionMatrixDisplay(confusion_matrix= confusionMatrix)
confusionMatrix.plot()
print("Precision:", precision_score(y_val, y_pred_val, average='macro'))
print("Recall:", recall_score(y_val, y_pred_val, average='macro'))

In [ ]:
# Test the classifier
y_pred_test = logistic_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Testing Accuracy:", test_accuracy)
print("Confusion Matrix:")
confusionMatrix = confusion_matrix(y_test, y_pred_test)
confusionMatrix = ConfusionMatrixDisplay(confusion_matrix= confusionMatrix)
confusionMatrix.plot()
print("Precision:", precision_score(y_test, y_pred_test, average='macro'))
print("Recall:", recall_score(y_test, y_pred_test, average='macro'))


In [ ]:
logistic_model = LogisticRegression(penalty="l2", C=0.001, solver='saga')
train_sizes = [0.01, 0.1, 0.25, 0.5, 0.8]
train_sizes, train_scores, val_scores = learning_curve(estimator=logistic_model, X=X_train, y=y_train, train_sizes=train_sizes, cv=5, shuffle=True)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, label='Training Score', color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Score', color='red')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.3, color='blue')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.3, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curve')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Create and train the logistic regression model
logistic_model = LogisticRegression(penalty='l2', C=0.01, solver='saga')
logistic_model.fit(X_train, y_train)

# Evaluate the model on the training set
train_predictions = logistic_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
train_error = 1 - train_accuracy

# Evaluate the model on the validation set
val_predictions = logistic_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
val_error = 1 - val_accuracy

# Evaluate the model on the test set
test_predictions = logistic_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
test_error = 1 - test_accuracy

# Plot the accuracy and error curves
x = ['Training', 'Validation', 'Test']
accuracy_values = [train_accuracy, val_accuracy, test_accuracy]
error_values = [train_error, val_error, test_error]

plt.figure(figsize=(8, 6))
plt.plot(x, accuracy_values, 'o-', color='b', label='Accuracy')
plt.plot(x, error_values, 'o-', color='r', label='Error')
plt.xlabel('Dataset')
plt.ylabel('Accuracy/Error')
plt.title('Accuracy and Error on Different Datasets')
plt.legend(loc='best')
plt.grid(True)
plt.show()


In [ ]:
# Interpreting Classifiers (example with feature importance for logistic regression)
feature_importance = np.abs(logistic_model.coef_[0])
sorted_indices = np.argsort(feature_importance)[::-1]
top_features = sorted_indices[:10] # Select top 10 important features

plt.figure(figsize=(8, 6))
plt.bar(range(10), feature_importance[top_features], align='center')
plt.xticks(range(10), top_features)
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.title('Feature Importance for Logistic Regression')
plt.grid(True)
plt.show()

In [ ]:
#Ensembles
voting_classifier = VotingClassifier([('logistic', logistic_model), ('random_forest', RandomForestClassifier())])
voting_classifier.fit(X_train, y_train)

y_pred_voting = voting_classifier.predict(X_test)
voting_accuracy = accuracy_score(y_test, y_pred_voting)
print("Voting Classifier Accuracy:", voting_accuracy)

In [ ]:
#Prediction Speed trained classifier
num_samples = 10000
X_test_speed = X_test[:num_samples]

start_time = time.time()
y_pred_speed = logistic_model.predict(X_test_speed)
end_time = time.time()

prediction_time = (end_time - start_time) * 1000 / num_samples
print("Prediction Speed (per example):", prediction_time, "ms")